# Complete Markowitz Analysis
This will complete the results under the following:

| | | | 
|----------------------------------------------------------------------|-----------------------------------------------|---------------------------------------------------------------------------------|
| n=4 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=4 $returns \sim N(\mu,\sigma)$ Independent  | n=4 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=8 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9  | n=8 $returns \sim N(\mu,\sigma)$ Independent  | n=8 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5)  |
| n=16 $returns \sim \mathcal{N}(\mu,\sigma)$ Correlation from 0.8-0.9 | n=16 $returns \sim N(\mu,\sigma)$ Independent | n=16 $returns \sim \mathcal{N}(\mu, sigma)$ Negative correlation (-0.8 to -0.5) |

using the method laid out in 4.1, Markowitz portfolio optimization

## Read Packages

In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
from pylab import rcParams
from mosek import iparam
import time
import random
import pypfopt as pyp
import sklearn.covariance
solvers.options['show_progress'] = False

In [15]:
import plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

## Simulate Data
Normal distribution with 0% correlation

Create function to generate random weights

In [16]:
def rand_weights(n):
    '''Produces n random weights that sum to 1'''
    k = np.random.rand(n)
    return k / sum(k)

Create function to generate random portfolio

In [17]:
def random_portfolio(returns):
    '''
    Returns the mean and standard deviation of returns for a randomly generated portfolio
    '''
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # Test for outliers to make plot prettier
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

Generates $n$ random portfolios

In [66]:
## This is a test - this doesn't need to run
#n_portfolios = 500
#means, stds = np.column_stack([
#    random_portfolio(return_vec)
#    for _ in range(n_portfolios)
#])

## Optimization

Create a series of functions that perform optimizations

In [50]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns + 5.0)
    
    N = 1000
    mus = [(1/n) + (t/N) for t in range(N)]#[10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis = 1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
    h = opt.matrix(0.0, (n,1))
    A = opt.matrix(1.0, (1,n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## Calculate risk and returns for frontier
    ret = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## Calculate the 2nd degree polynomail of the frontier curve
    m1 = np.polyfit(ret, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    ## Calculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    ret = np.zeros((n,1))
    rsk = np.zeros((n,1))
    for j in range(n):
        ret[j] = np.sum(np.array(wt)[j] * np.mean(returns, axis = 1)[j])
        rsk[j] = wt.T @ np.cov(returns) @ wt
    return np.asarray(wt), ret, rsk

In [8]:
# This one was a test - run previous cell
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns + 5)
    
    N = 1000
    mus = [np.ones(N)*(1/n) + 5.0]#[10**(5.0 * t/n - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis = 1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
    h = opt.matrix(0.0, (n,1))
    A = opt.matrix(1.0, (1,n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## Calculate risk and returns for frontier
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## Calculate the 2nd degree polynomail of the frontier curve
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    ## Calculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    ret = np.array([])
    rsk = np.array([])
    for j in range(n):
        ret[j] = 1/n * np.sum(wt[j] * np.asmatrix(returns)[:,j])
        rsk[j] = 1/N * np.sum(1/n * np.sum(np.asmatrix(risks)[:,j]))
    return np.asarray(wt), ret.T, risks

Create function that performs 1 optimization

In [19]:
def one_optimization(n_assets, n_obs):
    '''
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        return_vec = np.random.randn(n_assets, n_obs)# need to set the seeds here so others use the same values
        #means, stds = np.column_stack([
        #    random_portfolio(return_vec)
        #    for _ in range(n_portfolios)
        #    ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = returns.reshape(n_assets,)
        risks_res[i,:] = risks.reshape(n_assets,)
    return weight_res, return_res, risks_res

In [20]:
def one_optimization_zeros(n_assets, n_obs):
    '''
    This is exactly like one_optimization except it replaces 75% of asset 1 and 2 with zeros
    First, simulates portfolios then optimizes. 
    This does 100 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((999,n_assets))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        return_vec = np.random.randn(n_assets, n_obs)# need to set the seeds here so others use the same values
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[0,random.sample(list(range(n_obs)), _75_perct)] = 0
        np.random.seed(i*5)
        return_vec[1,random.sample(list(range(n_obs)), _75_perct)] = 0
        
        #means, stds = np.column_stack([
        #    random_portfolio(return_vec)
        #    for _ in range(n_portfolios)
        #    ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = returns.reshape(n_assets,)
        risks_res[i,:] = risks.reshape(n_assets,)
    return weight_res, return_res, risks_res

## Perform the optimizations specified

Each section performs an optimization for the selected number of assets with no zeros and then randomly selected zeros

## $n = 4$ 

In [51]:
start = time.time()
n4_normal_weight, n4_normal_return, n4_normal_risks = one_optimization(4, 1000)
end = time.time()
print(start - end)

-52.1361620426178


In [52]:
print('weights (mean) = '+str(np.round(np.mean(n4_normal_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_normal_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_normal_risks, axis=0),3)))

weights (mean) = [0.251 0.25  0.249 0.251] 
returns (mean) = [1.255 1.249 1.243 1.254] 
risks (mean) =[0.248 0.248 0.248 0.248]


In [53]:
print('weights (std) = '+str(np.round(np.std(n4_normal_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_normal_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_normal_risks, axis=0),3)))

weights (std) = [0.013 0.015 0.015 0.014] 
returns (std) = [0.068 0.079 0.076 0.074] 
risks (std) =[0.012 0.012 0.012 0.012]


In [54]:
n4_normal_weight_os, n4_normal_return_os, n4_normal_risks_os = one_optimization_zeros(4, 1000)

In [55]:
print('weights (mean) = '+str(np.round(np.mean(n4_normal_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_normal_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_normal_risks_os, axis=0),3)))

weights (mean) = [0.251 0.25  0.248 0.25 ] 
returns (mean) = [1.257 1.25  1.241 1.252] 
risks (mean) =[0.025 0.025 0.025 0.025]


In [56]:
print('weights (std) = '+str(np.round(np.std(n4_normal_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_normal_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_normal_risks_os, axis=0),3)))

weights (std) = [0.013 0.015 0.015 0.014] 
returns (std) = [0.068 0.079 0.076 0.074] 
risks (std) =[0.074 0.074 0.074 0.074]


## $n = 8$ 

In [57]:
n8_normal_weight, n8_normal_return, n8_normal_risks = one_optimization(8, 1000)

In [58]:
print('weights (mean) = '+str(np.round(np.mean(n8_normal_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_normal_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_normal_risks, axis=0),3)))

weights (mean) = [0.126 0.126 0.125 0.125 0.126 0.125 0.125 0.123] 
returns (mean) = [0.63  0.629 0.624 0.627 0.631 0.623 0.625 0.613] 
risks (mean) =[0.124 0.124 0.124 0.124 0.124 0.124 0.124 0.124]


In [59]:
print('weights (std) = '+str(np.round(np.std(n8_normal_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_normal_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_normal_risks, axis=0),3)))

weights (std) = [0.013 0.013 0.012 0.01  0.011 0.013 0.012 0.011] 
returns (std) = [0.065 0.067 0.062 0.054 0.059 0.068 0.064 0.056] 
risks (std) =[0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006]


In [60]:
n8_normal_weight_os, n8_normal_return_os, n8_normal_risks_os = one_optimization_zeros(8, 1000)

In [61]:
print('weights (mean) = '+str(np.round(np.mean(n8_normal_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_normal_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_normal_risks_os, axis=0),3)))

weights (mean) = [0.127 0.126 0.125 0.125 0.126 0.124 0.125 0.122] 
returns (mean) = [0.633 0.632 0.623 0.626 0.629 0.621 0.624 0.612] 
risks (mean) =[0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012]


In [62]:
print('weights (std) = '+str(np.round(np.std(n8_normal_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_normal_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_normal_risks_os, axis=0),3)))

weights (std) = [0.013 0.013 0.012 0.01  0.011 0.013 0.012 0.011] 
returns (std) = [0.065 0.067 0.062 0.054 0.059 0.068 0.064 0.056] 
risks (std) =[0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037]


## $n=16$

In [63]:
n16_normal_weight, n16_normal_return, n16_normal_risks = one_optimization(16, 1000)

In [64]:
print('weights (mean) = '+str(np.round(np.mean(n16_normal_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_normal_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_normal_risks, axis=0),3)))

weights (mean) = [0.063 0.063 0.063 0.063 0.063 0.063 0.064 0.061 0.062 0.064 0.061 0.061
 0.064 0.063 0.062 0.062] 
returns (mean) = [0.316 0.315 0.313 0.316 0.315 0.314 0.319 0.303 0.31  0.32  0.305 0.306
 0.319 0.316 0.308 0.309] 
risks (mean) =[0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062 0.062
 0.062 0.062 0.062 0.062]


In [65]:
print('weights (std) = '+str(np.round(np.std(n16_normal_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_normal_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_normal_risks, axis=0),3)))

weights (std) = [0.01  0.01  0.01  0.009 0.009 0.01  0.01  0.01  0.01  0.01  0.009 0.011
 0.01  0.011 0.01  0.009] 
returns (std) = [0.052 0.051 0.05  0.045 0.048 0.053 0.051 0.05  0.049 0.051 0.046 0.054
 0.05  0.055 0.05  0.045] 
risks (std) =[0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003 0.003
 0.003 0.003 0.003 0.003]


In [67]:
n16_normal_weight_os, n16_normal_return_os, n16_normal_risks_os = one_optimization_zeros(16,1000)

In [68]:
print('weights (mean) = '+str(np.round(np.mean(n16_normal_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_normal_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_normal_risks_os, axis=0),3)))

weights (mean) = [0.064 0.064 0.063 0.063 0.063 0.063 0.064 0.06  0.062 0.064 0.061 0.061
 0.064 0.063 0.062 0.062] 
returns (mean) = [0.319 0.318 0.313 0.316 0.314 0.314 0.318 0.302 0.309 0.32  0.304 0.305
 0.318 0.315 0.308 0.309] 
risks (mean) =[0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006 0.006
 0.006 0.006 0.006 0.006]


In [69]:
print('weights (std) = '+str(np.round(np.std(n16_normal_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_normal_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_normal_risks_os, axis=0),3)))

weights (std) = [0.01  0.01  0.01  0.009 0.009 0.01  0.01  0.01  0.01  0.01  0.009 0.011
 0.01  0.011 0.01  0.009] 
returns (std) = [0.052 0.052 0.05  0.045 0.048 0.052 0.051 0.05  0.049 0.051 0.047 0.054
 0.05  0.055 0.05  0.045] 
risks (std) =[0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019
 0.019 0.019 0.019 0.019]


## Play with Correlation

## Functions

In [71]:
def cor_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 30 replications
    n_assets: number of assets
    n_obs: number of observations
    r: covariance matrix
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs).T
        
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

In [72]:
def cor_optimization_zeros(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 30 replications
    n_assets: number of assets
    n_obs: number of observations
    r: covariance matrix
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_assets))
    risks_res = np.zeros((99,n_assets))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs).T
        _75_perct = int(len(return_vec[:,0])*3/4)
        return_vec[0,random.sample(list(range(1000)), _75_perct)] = 0
        np.random.seed(i*5)
        return_vec[1,random.sample(list(range(1000)), _75_perct)] = 0
        
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

## Results for Correlation between 0.8-0.9
Each section performs an optimization for the selected number of assets with no zeros and then randomly selected zeros

## $n=4$

In [73]:
# First, create a covariance structure you'd like to use, this one is 0.8-0.9
n = 4
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [74]:
n4_corr_weight, n4_corr_return, n4_corr_risks = cor_optimization(4, 1000, r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [75]:
print('weights (mean) = '+str(np.round(np.mean(n4_corr_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_corr_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_corr_risks, axis=0),3)))

weights (mean) = [0.274 0.104 0.199 0.423] 
returns (mean) = [1.37  0.522 0.993 2.113] 
risks (mean) =[0.878 0.878 0.878 0.878]


In [76]:
print('weights (std) = '+str(np.round(np.std(n4_corr_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_corr_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_corr_risks, axis=0),3)))

weights (std) = [0.073 0.068 0.062 0.062] 
returns (std) = [0.369 0.342 0.311 0.31 ] 
risks (std) =[0.037 0.037 0.037 0.037]


In [77]:
n4_corr_weight_os, n4_corr_return_os, n4_corr_risks_os = cor_optimization_zeros(4, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [79]:
print('weights (mean) = '+str(np.round(np.mean(n4_corr_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_corr_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_corr_risks_os, axis=0),3)))

weights (mean) = [0.27  0.099 0.22  0.41 ] 
returns (mean) = [1.351 0.494 1.102 2.05 ] 
risks (mean) =[0.876 0.876 0.876 0.876]


In [80]:
print('weights (std) = '+str(np.round(np.std(n4_corr_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_corr_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_corr_risks_os, axis=0),3)))

weights (std) = [0.066 0.056 0.064 0.056] 
returns (std) = [0.328 0.28  0.319 0.279] 
risks (std) =[0.038 0.038 0.038 0.038]


## $n = 8$

In [81]:
n = 8
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [82]:
n8_corr_weight, n8_corr_return, n8_corr_risks = cor_optimization(8, 1000, r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [83]:
print('weights (mean) = '+str(np.round(np.mean(n8_corr_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_corr_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_corr_risks, axis=0),3)))

weights (mean) = [0.227 0.159 0.027 0.113 0.138 0.06  0.182 0.094] 
returns (mean) = [1.135 0.794 0.136 0.565 0.692 0.299 0.911 0.469] 
risks (mean) =[0.855 0.855 0.855 0.855 0.855 0.855 0.855 0.855]


In [84]:
print('weights (std) = '+str(np.round(np.std(n8_corr_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_corr_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_corr_risks, axis=0),3)))

weights (std) = [0.071 0.071 0.04  0.071 0.071 0.059 0.064 0.065] 
returns (std) = [0.358 0.354 0.198 0.355 0.354 0.296 0.319 0.323] 
risks (std) =[0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036]


In [85]:
n8_corr_weight_os, n8_corr_return_os, n8_corr_risks_os = cor_optimization_zeros(8, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [86]:
print('weights (mean) = '+str(np.round(np.mean(n8_corr_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_corr_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_corr_risks_os, axis=0),3)))

weights (mean) = [0.237 0.153 0.021 0.119 0.124 0.053 0.192 0.101] 
returns (mean) = [1.186 0.765 0.107 0.596 0.621 0.263 0.961 0.506] 
risks (mean) =[0.851 0.851 0.851 0.851 0.851 0.851 0.851 0.851]


In [87]:
print('weights (std) = '+str(np.round(np.std(n8_corr_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_corr_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_corr_risks_os, axis=0),3)))

weights (std) = [0.073 0.066 0.032 0.072 0.068 0.065 0.07  0.059] 
returns (std) = [0.368 0.331 0.163 0.359 0.343 0.323 0.348 0.298] 
risks (std) =[0.042 0.042 0.042 0.042 0.042 0.042 0.042 0.042]


## $n=16$

In [88]:
n = 16
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [91]:
n16_corr_weight, n16_corr_return, n16_corr_risks = cor_optimization(16,1000,r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [92]:
print('weights (mean) = '+str(np.round(np.mean(n16_corr_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_corr_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_corr_risks, axis=0),3)))

weights (mean) = [0.065 0.052 0.111 0.03  0.055 0.071 0.019 0.079 0.051 0.128 0.041 0.028
 0.037 0.101 0.076 0.056] 
returns (mean) = [0.324 0.26  0.556 0.15  0.277 0.354 0.095 0.394 0.253 0.642 0.206 0.139
 0.187 0.503 0.379 0.28 ] 
risks (mean) =[0.837 0.837 0.837 0.837 0.837 0.837 0.837 0.837 0.837 0.837 0.837 0.837
 0.837 0.837 0.837 0.837]


In [93]:
print('weights (std) = '+str(np.round(np.std(n16_corr_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_corr_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_corr_risks, axis=0),3)))

weights (std) = [0.064 0.053 0.068 0.043 0.068 0.054 0.03  0.07  0.054 0.076 0.05  0.043
 0.048 0.063 0.058 0.059] 
returns (std) = [0.319 0.263 0.341 0.214 0.342 0.273 0.148 0.351 0.273 0.382 0.249 0.217
 0.238 0.316 0.291 0.297] 
risks (std) =[0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036 0.036
 0.036 0.036 0.036 0.036]


In [94]:
n16_corr_weight_os, n16_corr_return_os, n16_corr_risks_os = cor_optimization_zeros(16, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [95]:
print('weights (mean) = '+str(np.round(np.mean(n16_corr_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_corr_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_corr_risks_os, axis=0),3)))

weights (mean) = [0.094 0.077 0.114 0.029 0.073 0.072 0.014 0.065 0.04  0.11  0.041 0.026
 0.035 0.106 0.06  0.043] 
returns (mean) = [0.471 0.383 0.569 0.147 0.366 0.359 0.068 0.327 0.202 0.55  0.204 0.13
 0.177 0.528 0.301 0.216] 
risks (mean) =[0.846 0.846 0.846 0.846 0.846 0.846 0.846 0.846 0.846 0.846 0.846 0.846
 0.846 0.846 0.846 0.846]


In [96]:
print('weights (std) = '+str(np.round(np.std(n16_corr_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_corr_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_corr_risks_os, axis=0),3)))

weights (std) = [0.067 0.059 0.066 0.042 0.065 0.06  0.026 0.061 0.047 0.073 0.051 0.048
 0.047 0.068 0.057 0.049] 
returns (std) = [0.335 0.297 0.331 0.21  0.325 0.298 0.128 0.307 0.233 0.368 0.255 0.24
 0.233 0.342 0.286 0.245] 
risks (std) =[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
 0.04 0.04]


## Results for Correlation between -0.6 and -0.5

## $n=4$

In [97]:
n = 4
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [98]:
n4_neg_weight, n4_neg_return, n4_neg_risks = cor_optimization(4, 1000, r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [99]:
print('weights (mean) = '+str(np.round(np.mean(n4_neg_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_neg_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_neg_risks, axis=0),3)))

weights (mean) = [0.237 0.286 0.266 0.21 ] 
returns (mean) = [1.186 1.431 1.332 1.05 ] 
risks (mean) =[0.232 0.232 0.232 0.232]


In [100]:
print('weights (std) = '+str(np.round(np.std(n4_neg_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_neg_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_neg_risks, axis=0),3)))

weights (std) = [0.011 0.011 0.011 0.01 ] 
returns (std) = [0.06  0.057 0.058 0.054] 
risks (std) =[0.011 0.011 0.011 0.011]


In [101]:
n4_neg_weight_os, n4_neg_return_os, n4_neg_risks_os = cor_optimization_zeros(4, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [102]:
print('weights (mean) = '+str(np.round(np.mean(n4_neg_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n4_neg_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n4_neg_risks_os, axis=0),3)))

weights (mean) = [0.237 0.289 0.262 0.212] 
returns (mean) = [1.185 1.446 1.31  1.058] 
risks (mean) =[0.231 0.231 0.231 0.231]


In [103]:
print('weights (std) = '+str(np.round(np.std(n4_neg_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n4_neg_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n4_neg_risks_os, axis=0),3)))

weights (std) = [0.011 0.012 0.011 0.011] 
returns (std) = [0.06  0.065 0.057 0.058] 
risks (std) =[0.009 0.009 0.009 0.009]


## $n=8$

In [105]:
n = 8
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [106]:
n8_neg_weight, n8_neg_return, n8_neg_risks = cor_optimization(8, 1000, r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [107]:
print('weights (mean) = '+str(np.round(np.mean(n8_neg_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_neg_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_neg_risks, axis=0),3)))

weights (mean) = [0.104 0.117 0.141 0.13  0.122 0.143 0.114 0.129] 
returns (mean) = [0.521 0.585 0.706 0.65  0.61  0.715 0.569 0.645] 
risks (mean) =[0.448 0.448 0.448 0.448 0.448 0.448 0.448 0.448]


In [108]:
print('weights (std) = '+str(np.round(np.std(n8_neg_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_neg_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_neg_risks, axis=0),3)))

weights (std) = [0.017 0.017 0.017 0.018 0.017 0.015 0.015 0.015] 
returns (std) = [0.084 0.087 0.085 0.093 0.086 0.073 0.076 0.078] 
risks (std) =[0.019 0.019 0.019 0.019 0.019 0.019 0.019 0.019]


In [109]:
n8_neg_weight_os, n8_neg_return_os, n8_neg_risks_os = cor_optimization_zeros(8, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [110]:
print('weights (mean) = '+str(np.round(np.mean(n8_neg_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n8_neg_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n8_neg_risks_os, axis=0),3)))

weights (mean) = [0.106 0.119 0.142 0.128 0.119 0.143 0.114 0.129] 
returns (mean) = [0.532 0.595 0.709 0.642 0.594 0.713 0.57  0.647] 
risks (mean) =[0.449 0.449 0.449 0.449 0.449 0.449 0.449 0.449]


In [111]:
print('weights (std) = '+str(np.round(np.std(n8_neg_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n8_neg_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n8_neg_risks_os, axis=0),3)))

weights (std) = [0.014 0.016 0.014 0.018 0.013 0.014 0.016 0.016] 
returns (std) = [0.068 0.083 0.072 0.092 0.068 0.073 0.083 0.079] 
risks (std) =[0.018 0.018 0.018 0.018 0.018 0.018 0.018 0.018]


## $n=16$

In [112]:
n = 16
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [113]:
n16_neg_weight, n16_neg_return, n16_neg_risks = cor_optimization(16, 1000, r)

<ipython-input-71-6afc58b1f877>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [114]:
print('weights (mean) = '+str(np.round(np.mean(n16_neg_weight, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_neg_return, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_neg_risks, axis=0),3)))

weights (mean) = [0.064 0.071 0.05  0.072 0.052 0.058 0.077 0.058 0.069 0.041 0.068 0.071
 0.072 0.045 0.064 0.067] 
returns (mean) = [0.32  0.353 0.252 0.361 0.262 0.29  0.386 0.291 0.347 0.204 0.342 0.357
 0.359 0.226 0.319 0.334] 
risks (mean) =[0.543 0.543 0.543 0.543 0.543 0.543 0.543 0.543 0.543 0.543 0.543 0.543
 0.543 0.543 0.543 0.543]


In [115]:
print('weights (std) = '+str(np.round(np.std(n16_neg_weight, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_neg_return, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_neg_risks, axis=0),3)))

weights (std) = [0.019 0.019 0.018 0.018 0.018 0.019 0.018 0.018 0.018 0.018 0.016 0.018
 0.019 0.018 0.02  0.018] 
returns (std) = [0.093 0.098 0.092 0.09  0.092 0.094 0.09  0.092 0.093 0.092 0.079 0.091
 0.098 0.088 0.101 0.09 ] 
risks (std) =[0.027 0.027 0.027 0.027 0.027 0.027 0.027 0.027 0.027 0.027 0.027 0.027
 0.027 0.027 0.027 0.027]


In [116]:
n16_neg_weight_os, n16_neg_return_os, n16_neg_risks_os = cor_optimization_zeros(16, 1000, r)

<ipython-input-72-6eda513a3af1>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [117]:
print('weights (mean) = '+str(np.round(np.mean(n16_neg_weight_os, axis=0),3)),
      '\nreturns (mean) = '+str(np.round(np.mean(n16_neg_return_os, axis=0),3)),
      '\nrisks (mean) ='+str(np.round(np.mean(n16_neg_risks_os, axis=0),3)))

weights (mean) = [0.064 0.073 0.051 0.075 0.053 0.059 0.075 0.056 0.068 0.041 0.066 0.075
 0.071 0.046 0.061 0.066] 
returns (mean) = [0.319 0.366 0.254 0.375 0.268 0.297 0.374 0.282 0.338 0.205 0.328 0.377
 0.355 0.228 0.306 0.333] 
risks (mean) =[0.539 0.539 0.539 0.539 0.539 0.539 0.539 0.539 0.539 0.539 0.539 0.539
 0.539 0.539 0.539 0.539]


In [118]:
print('weights (std) = '+str(np.round(np.std(n16_neg_weight_os, axis=0),3)),
      '\nreturns (std) = '+str(np.round(np.std(n16_neg_return_os, axis=0),3)),
      '\nrisks (std) ='+str(np.round(np.std(n16_neg_risks_os, axis=0),3)))

weights (std) = [0.019 0.018 0.02  0.015 0.017 0.02  0.019 0.018 0.018 0.019 0.018 0.015
 0.016 0.018 0.019 0.019] 
returns (std) = [0.094 0.09  0.098 0.078 0.085 0.098 0.094 0.09  0.089 0.096 0.09  0.077
 0.083 0.092 0.097 0.097] 
risks (std) =[0.026 0.026 0.026 0.026 0.026 0.026 0.026 0.026 0.026 0.026 0.026 0.026
 0.026 0.026 0.026 0.026]


In [104]:
## I was testing something out here, but decided to abandon it
#returns = np.random.randn(n_assets, n_obs)
#n = len(returns)
#returns = np.asmatrix(returns + 5)
    
#S = np.array(np.cov(returns))
#pbar = np.array(np.mean(returns, axis = 1))
    
#ef = pyp.EfficientFrontier(pbar, S)
#ef.add_constraint(lambda w: sum(w) == 1)
#ef.efficient_return(0.25)